# Training Data without adding number of sequnces

!pip install tensorflow-addons==0.16.1

In [1]:
# !pip install tensorflow-addons==0.16.1
import math
import numpy as np
import pandas as pd
import tensorflow as tf
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape,TimeDistributed,Input,LSTM,Activation,Dense,BatchNormalization, LeakyReLU, Conv2D, GlobalAveragePooling1D,MaxPooling2D, MaxPooling1D, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adadelta, Adagrad
from tensorflow_addons.optimizers import AdamW
from tensorflow_addons.layers import InstanceNormalization
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from numpy import zeros, newaxis
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import mutual_info_regression,SelectKBest
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import callbacks
from keras.initializers import random_uniform
np.set_printoptions(suppress=True)
from keras.callbacks import LearningRateScheduler
%matplotlib inline
from ResNet_1DCNN import ResNet

C:\Users\user\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
## Load Data
Insole = pd.read_csv('1225JakariaRWalk5Min.txt', header=None, low_memory=False)
SIData =  np.array(Insole)

df = pd.read_csv('1225JakariaRWalk5Min.csv', low_memory=False)
columns = ['Fx','Fy','Fz','Mx','My','Mz']
selected_df = df[columns]
FPDatas = selected_df[:15000]

SmartInsole = np.array(SIData[:15000]).astype('float64')
FPData = np.array(FPDatas).astype('float64')

FXData = FPData[:,0]
FXData =  np.array(FXData)
FXData = FXData.reshape(-1,1)
FYData = FPData[:,1]
FYData =  np.array(FYData)
FYData = FYData.reshape(-1,1)
FZData = (FPData[:,2])/10
FZData =  np.array(FZData)
FZData = FZData.reshape(-1,1)
MXData = (FPData[:,3])/1000
MXData =  np.array(MXData)
MXData = MXData.reshape(-1,1)
MYData = (FPData[:,4])/10000
MYData =  np.array(MYData)
MYData = MYData.reshape(-1,1)
MZData = (FPData[:,5])/100
MZData =  np.array(MZData)
MZData = MZData.reshape(-1,1)

newFPData = np.concatenate((FXData, FYData, FZData, MXData, MYData, MZData), axis=1)
## End Load Data

In [3]:
# # Jittering by adding random noise
# n = 50

# # create n random values
# noise = np.random.normal(loc=0, scale=0.1, size=n)
# noise = np.array(noise)
# for i in range(n):
#   noise[i] = noise[i] + i

# # loop through rows in steps of n
# for i in range(0, SmartInsole.shape[0], n):
#     SmartInsole[i:i + n, 0] = np.clip(SmartInsole[i:i + 1, 0] + noise, a_min=0.0, a_max=None)

max_iter = 50
iter = 0
for i in range(len(SmartInsole)):
    SmartInsole[i][0] = SmartInsole[i][0] + (iter % max_iter) + 1
    iter += 1

In [4]:
# Data Normalization
minInsole = SmartInsole.min()
maxInsole = SmartInsole.max()
xscale = (SmartInsole - minInsole) / ( maxInsole - minInsole )

FPmax = []
FPmin = []
yscale = []

for i in range(0,6):
    minFP = FPData[:,i].min()
    maxFP = FPData[:,i].max()
    FPmin.append(minFP)
    FPmax.append(maxFP)

FPmin = np.array(FPmin)
FPmax = np.array(FPmax)

for i in range(0,6):
  scale = (FPData[:,i] - FPmin[i]) / ( FPmax[i] - FPmin[i] )
  yscale.append(scale)
yscale = np.array(yscale)
yscale = yscale.transpose()
#End Data Normalization

In [5]:
#Spliting Data
sample_size = xscale.shape[0] # number of samples in train set
time_steps  = xscale.shape[1] # number of features in train set
input_dimension = 1               # each feature is represented by 1 number

train_data_reshaped = xscale.reshape(sample_size,time_steps,input_dimension)

X_train, X_test, y_train, y_test = train_test_split(train_data_reshaped, yscale, test_size=0.20, random_state=2)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)
#End Spliting Data

(12000, 89, 1) (3000, 89, 1)
(12000, 6) (3000, 6)


In [6]:
"Configurations for ResNet in Regression Mode"
length = X_train.shape[1]   # Number of Features (or length of the signal)
model_width = 64           # Number of Filter or Kernel in the Input Layer
num_channel = 1             # Number of Input Channels
problem_type = 'Regression' # Regression or Classification
output_number = 6           # Number of Outputs in the Regression Mode

In [7]:
Regression_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).ResNet18() # Build Model
# ResNet Models supported: ResNet18, ResNet34, ResNet50, ResNet101, ResNet152, 
Regression_Model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001), metrics= ['mse']) # Compile Model
# Here, Model validation metric is set as Mean Squared Error or MSE

In [8]:
Regression_Model.summary() # Summary of the Model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 89, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 45, 64)       512         ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 45, 64)       0           ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 22, 64)       0           ['activation[0][0]']             
                                                                                              

 conv1d_14 (Conv1D)             (None, 3, 512)       786944      ['activation_17[0][0]']          
                                                                                                  
 activation_18 (Activation)     (None, 3, 512)       0           ['conv1d_14[0][0]']              
                                                                                                  
 conv1d_15 (Conv1D)             (None, 3, 512)       786944      ['activation_18[0][0]']          
                                                                                                  
 activation_19 (Activation)     (None, 3, 512)       0           ['conv1d_15[0][0]']              
                                                                                                  
 conv1d_16 (Conv1D)             (None, 3, 512)       786944      ['activation_19[0][0]']          
                                                                                                  
 activatio

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=30, mode='min'), ModelCheckpoint('1Person_Walking_ResNet18.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
history = Regression_Model.fit(X_train, y_train, epochs=500, batch_size=64,
                               verbose=1, validation_split=0.2, shuffle=True)


# history = Regression_Model.fit(X_train, y_train, batch_size=64, epochs=500,
#                     validation_data=(X_test, y_test), verbose=2,
#                     callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=30,verbose=2, mode='auto')])


Epoch 1/500
150/150 [==============================] - 4s 14ms/step - loss: 0.1946 - mse: 0.1946 - val_loss: 0.0207 - val_mse: 0.0207
Epoch 2/500
150/150 [==============================] - 2s 12ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.0206 - val_mse: 0.0206
Epoch 3/500
150/150 [==============================] - 2s 12ms/step - loss: 0.0191 - mse: 0.0191 - val_loss: 0.0171 - val_mse: 0.0171
Epoch 4/500
150/150 [==============================] - 2s 12ms/step - loss: 0.0175 - mse: 0.0175 - val_loss: 0.0185 - val_mse: 0.0185
Epoch 5/500
150/150 [==============================] - 2s 12ms/step - loss: 0.0153 - mse: 0.0153 - val_loss: 0.0133 - val_mse: 0.0133
Epoch 6/500
150/150 [==============================] - 2s 12ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 7/500
150/150 [==============================] - 2s 12ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.0088 - val_mse: 0.0088
Epoch 8/500
150/150 [==============================] - 2s 12ms

In [ ]:
#Evaluate Model
Regression_Model.evaluate(train_data_reshaped, yscale)
ypred = Regression_Model.predict(train_data_reshaped)

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
# plt.show()
plt.savefig('Loss Result.png')

print('MSE: ',mean_squared_error(yscale, ypred))
print('RMSE: ',math.sqrt(mean_squared_error(yscale, ypred)))
print('Coefficient of determination (r2 Score): ', r2_score(yscale, ypred))


#Inverse
y_inverse = []
y_pred_inverse = []

for i in range(0,6):
  Y_inver =  yscale[0:15000, i]*( FPmax[i] - FPmin[i] )+FPmin[i]
  Pred_inver = ypred[0:15000, i]*( FPmax[i] - FPmin[i] )+FPmin[i]
  y_inverse.append(Y_inver)
  y_pred_inverse.append(Pred_inver)
y_inverse = np.array(y_inverse)
y_inverse = y_inverse.transpose()
y_pred_inverse = np.array(y_pred_inverse)
y_pred_inverse = y_pred_inverse.transpose()

for i in range(len(y_pred_inverse)):
    if (np.abs(y_pred_inverse[i]) < 1).all():
        y_pred_inverse[i] = 0

print('MSE: ',mean_squared_error(y_inverse, y_pred_inverse))
print('RMSE: ',math.sqrt(mean_squared_error(y_inverse, y_pred_inverse)))
print('Coefficient of determination (r2 Score): ', r2_score(y_inverse, y_pred_inverse))

# restore to original Data
# FXData2 = y_inverse[:,0]
# FXData2=  np.array(FXData2)
# FXData2 = FXData2.reshape(-1,1)

# FYData2 = y_inverse[:,1]
# FYData2 =  np.array(FYData2)
# FYData2 = FYData2.reshape(-1,1)

# FZData2 = (y_inverse[:,2]*10)
# FZData2 =  np.array(FZData2)
# FZData2 = FZData2.reshape(-1,1)

# MXData2 = (y_inverse[:,3]*1000)
# MXData2 =  np.array(MXData2)
# MXData2 = MXData2.reshape(-1,1)

# MYData2 = (y_inverse[:,4]*10000)
# MYData2 =  np.array(MYData2)
# MYData2 = MYData2.reshape(-1,1
#                           )
# MZData2 = (y_inverse[:,5]*100)
# MZData2 =  np.array(MZData2)
# MZData2 = MZData2.reshape(-1,1)

# new_inverse2 = np.concatenate((FXData2, FYData2, FZData2, MXData2, MYData2, MZData2), axis=1)

# FXData3 = y_pred_inverse[:,0]
# FXData3=  np.array(FXData3)
# FXData3 = FXData3.reshape(-1,1)

# FYData3 = y_pred_inverse[:,1]
# FYData3 =  np.array(FYData3)
# FYData3 = FYData3.reshape(-1,1)

# FZData3 = (y_pred_inverse[:,2]*10)
# FZData3 =  np.array(FZData3)
# FZData3 = FZData3.reshape(-1,1)

# MXData3 = (y_pred_inverse[:,3]*1000)
# MXData3 =  np.array(MXData3)
# MXData3 = MXData3.reshape(-1,1)

# MYData3 = (y_pred_inverse[:,4]*10000)
# MYData3 =  np.array(MYData3)
# MYData3 = MYData3.reshape(-1,1)

# MZData3 = (y_pred_inverse[:,5]*100)
# MZData3 =  np.array(MZData3)
# MZData3 = MZData3.reshape(-1,1)

# new_inverse3 = np.concatenate((FXData3, FYData3, FZData3, MXData3, MYData3, MZData3), axis=1)

new_inverse2 = y_inverse
new_inverse3 = y_pred_inverse

x=[]
colors=['red','green','brown','teal','gray','black','maroon','orange','purple']
colors2=['green','red','orange','black','maroon','teal','blue','gray','brown']
x = np.arange(0,3000)*60/3000
for i in range(0,6):
    plt.figure(figsize=(15,6))
    # plt.figure()
    plt.plot(x,new_inverse2[0:3000,i],color='red')
    plt.plot(x,new_inverse3[0:3000,i], markerfacecolor='none',color='green')
    plt.title('CNN Regression (Training Data)')
    if i < 3:
      plt.ylabel('Force/'+columns[i])
    else:
      plt.ylabel('Moment/'+columns[i])
    plt.xlabel('Time(s)')
    plt.legend(['Real value', 'Predicted Value'], loc='best')
    plt.show()

# COP
from math import*
np.seterr(divide='ignore', invalid='ignore')

out_Fz = new_inverse2[0:15000,2]
out_Mx = new_inverse2[0:15000,3]
out_My = new_inverse2[0:15000,4]
Pred_Fz = new_inverse3[0:15000,2]
Pred_Mx = new_inverse3[0:15000,3]
Pred_My = new_inverse3[0:15000,4]

Pred_COPx=[]
for i in range(0,len(Pred_Fz)):
  Pred_COPx_temp=-(Pred_My[i])/Pred_Fz[i]
  # print(temp)
  if Pred_COPx_temp != Pred_COPx_temp:
    Pred_COPx_temp=0
  Pred_COPx.append(Pred_COPx_temp)
  # break

out_COPx=[]
for i in range(0,len(out_Fz)):
  out_COPx_temp=-(out_My[i])/out_Fz[i]
  # print(temp)
  if out_COPx_temp != out_COPx_temp:
    out_COPx_temp=0
  out_COPx.append(out_COPx_temp)
  # break

Pred_COPy=[]
for i in range(0,len(Pred_Mx)):
  Pred_COPy_temp=Pred_Mx[i]/Pred_Fz[i]
  # print(temp)
  if Pred_COPy_temp != Pred_COPy_temp:
    Pred_COPy_temp=0
  Pred_COPy.append(Pred_COPy_temp)
  # break

out_COPy=[]
for i in range(0,len(out_Mx)):
  out_COPy_temp=out_Mx[i]/out_Fz[i]
  # print(temp)
  if out_COPy_temp != out_COPy_temp:
    out_COPy_temp=0
  out_COPy.append(out_COPy_temp)
  # break


# out_COPx = -(out_My)/out_Fz
out_COPx = np.array(out_COPx)
out_COPx= out_COPx.reshape(-1,1)

# out_COPy = out_Mx/out_Fz
out_COPy = np.array(out_COPy)
out_COPy= out_COPy.reshape(-1,1)

# Pred_COPx = -(Pred_My)/Pred_Fz
Pred_COPx = np.array(Pred_COPx)
Pred_COPx= Pred_COPx.reshape(-1,1)

# Pred_COPy = Pred_Mx/Pred_Fz
Pred_COPy = np.array(Pred_COPy)
Pred_COPy= Pred_COPy.reshape(-1,1)

Pred_COP = np.concatenate((Pred_COPx, Pred_COPy), axis=1)
FC_COP = np.concatenate((out_COPx, out_COPy), axis=1)

col_COP = 'COPx', 'COPy'

x=[]
colors=['red','green','brown','teal','gray','black','maroon','orange','purple']
colors2=['green','red','orange','black','maroon','teal','blue','gray','brown']
x = np.arange(0,2000)*40/2000
for i in range(0,2):
    plt.figure(figsize=(15,6))
    plt.plot(x,FC_COP[0:2000,i], color='red')
    plt.plot(x,Pred_COP[0:2000,i],markerfacecolor='none',color='green')
    plt.title('COP Calculation (Training Data)')
    plt.ylabel(col_COP[i])
    plt.xlabel('Time(s)')
    plt.legend(['Groundtruth', 'Predicted Value'], loc='best')
    plt.savefig('Regression Result.png'[i])
    plt.show()

# Trajectory
from matplotlib import pyplot

x = range(50)
y1 = FC_COP[50:100,0]
y2 = FC_COP[50:100,1]
y3 = Pred_COP[50:100,0]
y4 = Pred_COP[50:100,1]

# pyplot.figure(figsize=(15,6))
# pyplot.plot(FC_COP[:,0],FC_COP[:,1])
# pyplot.show()

data_filter = abs(y1) > 0
data_filter2 = abs(y3) > 0
pyplot.figure(figsize=(15,6))
pyplot.plot(y1[data_filter], y2[data_filter ], color='red', alpha=0.3)
pyplot.plot(y3[data_filter2], y4[data_filter2 ], color='green')
# pyplot.plot(y1, y2, color='red')
# pyplot.plot(y3, y4, color='green')
plt.title('COP Trajectory (Training Data)')
pyplot.ylabel('COPy (mm)')
pyplot.xlabel('COPx (mm)')
pyplot.legend(['Groundtruth', 'Predicted'], loc='best')
pyplot.show()

In [ ]:
print(y_inverse[0],y_pred_inverse[0])

In [ ]:
x = range(100)
y1 = FC_COP[100:200,0]
y2 = FC_COP[100:200,1]
y3 = Pred_COP[100:200,0]
y4 = Pred_COP[100:200,1]

pyplot.figure(figsize=(15,6))
pyplot.plot(y1, y2, color='red')
pyplot.plot(y3, y4, color='green')
plt.title('COP Trajectory (Training Data)')
pyplot.ylabel('COPy (mm)')
pyplot.xlabel('COPx (mm)')
pyplot.legend(['Groundtruth', 'Predicted'], loc='best')
pyplot.show()

# Testing

In [ ]:
## Model Validation
Test_Insole = pd.read_csv('1225JakariaRWalk2Min.txt', header=None, low_memory=False)
TestSIData =  np.asarray(Test_Insole)

Test_df = pd.read_csv('1225JakariaRWalk2Min.csv', low_memory=False)
Test_columns = ['Fx','Fy','Fz','Mx','My','Mz']
Test_selected_df = Test_df[Test_columns]
Test_FPDatas = Test_selected_df[:4200]


test_SmartInsole = np.array(TestSIData[:4200]).astype('float32')
Test_FPData = np.array(Test_FPDatas).astype('float32')

Test_FXData = Test_FPData[:,0]
Test_FXData =  np.array(Test_FXData)
Test_FXData = Test_FXData.reshape(-1,1)
Test_FYData = Test_FPData[:,1]
Test_FYData =  np.array(Test_FYData)
Test_FYData = Test_FYData.reshape(-1,1)
Test_FZData = (Test_FPData[:,2])/10
Test_FZData =  np.array(Test_FZData)
Test_FZData = Test_FZData.reshape(-1,1)
Test_MXData = (Test_FPData[:,3])/1000
Test_MXData =  np.array(Test_MXData)
Test_MXData = Test_MXData.reshape(-1,1)
Test_MYData = (Test_FPData[:,4])/10000
Test_MYData =  np.array(Test_MYData)
Test_MYData =Test_MYData.reshape(-1,1)
Test_MZData = (Test_FPData[:,5])/100
Test_MZData =  np.array(Test_MZData)
Test_MZData = Test_MZData.reshape(-1,1)

Test_newFPData = np.concatenate((Test_FXData, Test_FYData, Test_FZData, Test_MXData, Test_MYData, Test_MZData), axis=1)
## End Load Data

# loop through rows in steps of n
# for i in range(0, test_SmartInsole.shape[0], n):
#     test_SmartInsole[i:i + n, 0] = np.clip(test_SmartInsole[i:i + 1, 0] + noise, a_min=0.0, a_max=None)

for i in range(len(test_SmartInsole)):
    test_SmartInsole[i][0] = test_SmartInsole[i][0] + (iter % max_iter) + 1
    iter += 1

Test_xscale = (test_SmartInsole - minInsole) / ( maxInsole - minInsole )

Test_yscale = []
for i in range(0,6):
  Test_scale = (Test_FPData[:,i] - FPmin[i]) / ( FPmax[i] - FPmin[i] )
  Test_yscale.append(Test_scale)
Test_yscale = np.array(Test_yscale)
Test_yscale = Test_yscale.transpose()

test_sample_size = Test_xscale.shape[0] # number of samples in train set
test_time_steps  = Test_xscale.shape[1] # number of features in train set
test_input_dimension = 1               # each feature is represented by 1 number

test_train_data_reshaped = Test_xscale.reshape(test_sample_size,test_time_steps,test_input_dimension)

Regression_Model.evaluate(test_train_data_reshaped, Test_yscale)
Test_xX_model = Regression_Model.predict(test_train_data_reshaped)

print('MSE: ',mean_squared_error(Test_yscale, ypred))
print('RMSE: ',math.sqrt(mean_squared_error(Test_yscale, ypred)))
print('Coefficient of determination (r2 Score): ', r2_score(Test_yscale, ypred))


#invert normalize
Test_y_inverse = []
Test_y_pred_inverse = []

for i in range(0,6):
  Test_Y_inver =  Test_yscale[0:15000, i]*( FPmax[i] - FPmin[i] )+FPmin[i]
  Test_Pred_inver = Test_xX_model[0:15000, i]*( FPmax[i] - FPmin[i] )+FPmin[i]
  Test_y_inverse.append(Test_Y_inver)
  Test_y_pred_inverse.append(Test_Pred_inver)
Test_y_inverse = np.array(Test_y_inverse)
Test_y_inverse = Test_y_inverse.transpose()
Test_y_pred_inverse = np.array(Test_y_pred_inverse)
Test_y_pred_inverse = Test_y_pred_inverse.transpose()

for i in range(len(Test_y_pred_inverse)):
    if (np.abs(Test_y_pred_inverse[i]) < 1).all():
        Test_y_pred_inverse[i] = 0

# # make to original Data
# Test_FXData2 = Test_y_inverse[:,0]
# Test_FXData2=  np.array(Test_FXData2)
# Test_FXData2 = Test_FXData2.reshape(-1,1)

# Test_FYData2 = Test_y_inverse[:,1]
# Test_FYData2 =  np.array(Test_FYData2)
# Test_FYData2 = Test_FYData2.reshape(-1,1)

# Test_FZData2 = (Test_y_inverse[:,2]*10)
# Test_FZData2 =  np.array(Test_FZData2)
# Test_FZData2 = Test_FZData2.reshape(-1,1)

# Test_MXData2 = (Test_y_inverse[:,3]*1000)
# Test_MXData2 =  np.array(Test_MXData2)
# Test_MXData2 = Test_MXData2.reshape(-1,1)

# Test_MYData2 = (Test_y_inverse[:,4]*10000)
# Test_MYData2 =  np.array(Test_MYData2)
# Test_MYData2 = Test_MYData2.reshape(-1,1
#                           )
# Test_MZData2 = (Test_y_inverse[:,5]*100)
# Test_MZData2 =  np.array(Test_MZData2)
# Test_MZData2 = Test_MZData2.reshape(-1,1)

# Test_new_inverse2 = np.concatenate((Test_FXData2, Test_FYData2, Test_FZData2, Test_MXData2, Test_MYData2, Test_MZData2), axis=1)

# Test_FXData3 = Test_y_pred_inverse[:,0]
# Test_FXData3=  np.array(Test_FXData3)
# Test_FXData3 = Test_FXData3.reshape(-1,1)

# Test_FYData3 = Test_y_pred_inverse[:,1]
# Test_FYData3 =  np.array(Test_FYData3)
# Test_FYData3 = Test_FYData3.reshape(-1,1)

# Test_FZData3 = (Test_y_pred_inverse[:,2]*10)
# Test_FZData3 =  np.array(Test_FZData3)
# Test_FZData3 = Test_FZData3.reshape(-1,1)

# Test_MXData3 = (Test_y_pred_inverse[:,3]*1000)
# Test_MXData3 =  np.array(Test_MXData3)
# Test_MXData3 = Test_MXData3.reshape(-1,1)

# Test_MYData3 = (Test_y_pred_inverse[:,4]*10000)
# Test_MYData3 =  np.array(Test_MYData3)
# Test_MYData3 = Test_MYData3.reshape(-1,1)

# Test_MZData3 = (Test_y_pred_inverse[:,5]*100)
# Test_MZData3 =  np.array(Test_MZData3)
# Test_MZData3 = Test_MZData3.reshape(-1,1)

# Test_new_inverse3 = np.concatenate((Test_FXData3, Test_FYData3, Test_FZData3, Test_MXData3, Test_MYData3, Test_MZData3), axis=1)

Test_new_inverse2 = Test_y_inverse
Test_new_inverse3 = Test_y_pred_inverse

x=[]
colors=['red','green','brown','teal','gray','black','maroon','orange','purple']
colors2=['green','red','orange','black','maroon','teal','blue','gray','brown']
x = np.arange(0,3000)*60/3000
for i in range(0,6):
    plt.figure(figsize=(15,6))
    # plt.figure()
    plt.plot(x,Test_new_inverse2[0:3000,i],color='red')
    plt.plot(x,Test_new_inverse3[0:3000,i], markerfacecolor='none',color='green')
    plt.title('CNN Regression (Testing Data)')
    if i < 3:
      plt.ylabel('Force/'+columns[i])
    else:
      plt.ylabel('Moment/'+columns[i])
    plt.xlabel('Time(s)')
    plt.legend(['Groundtruth', 'Predicted'], loc='best')
    plt.show()

# COP
from math import*
np.seterr(divide='ignore', invalid='ignore')

Test_out_Fz = Test_new_inverse2[0:15000,2]
Test_out_Mx = Test_new_inverse2[0:15000,3]
Test_out_My = Test_new_inverse2[0:15000,4]
Test_Pred_Fz = Test_new_inverse3[0:15000,2]
Test_Pred_Mx = Test_new_inverse3[0:15000,3]
Test_Pred_My = Test_new_inverse3[0:15000,4]

Test_Pred_COPx=[]
for i in range(0,len(Test_Pred_Fz)):
  Test_Pred_COPx_temp=-(Test_Pred_My[i])/Test_Pred_Fz[i]
  if Test_Pred_COPx_temp != Test_Pred_COPx_temp:
    Test_Pred_COPx_temp=0
  Test_Pred_COPx.append(Test_Pred_COPx_temp)
  # break

Test_out_COPx=[]
for i in range(0,len(Test_out_Fz)):
  Test_out_COPx_temp=-(Test_out_My[i])/Test_out_Fz[i]
  if Test_out_COPx_temp != Test_out_COPx_temp:
    Test_out_COPx_temp=0
  Test_out_COPx.append(Test_out_COPx_temp)
  # break

Test_Pred_COPy=[]
for i in range(0,len(Test_Pred_Mx)):
  Test_Pred_COPy_temp=Test_Pred_Mx[i]/Test_Pred_Fz[i]
  if Test_Pred_COPy_temp != Test_Pred_COPy_temp:
    Test_Pred_COPy_temp=0
  Test_Pred_COPy.append(Test_Pred_COPy_temp)
  # break

Test_out_COPy=[]
for i in range(0,len(Test_out_Mx)):
  Test_out_COPy_temp=Test_out_Mx[i]/Test_out_Fz[i]
  if Test_out_COPy_temp != Test_out_COPy_temp:
    Test_out_COPy_temp=0
  Test_out_COPy.append(Test_out_COPy_temp)
  # break


Test_out_COPx = np.array(Test_out_COPx)
Test_out_COPx= Test_out_COPx.reshape(-1,1)

Test_out_COPy = np.array(Test_out_COPy)
Test_out_COPy= Test_out_COPy.reshape(-1,1)

Test_Pred_COPx = np.array(Test_Pred_COPx)
Test_Pred_COPx= Test_Pred_COPx.reshape(-1,1)

Test_Pred_COPy = np.array(Test_Pred_COPy)
Test_Pred_COPy= Test_Pred_COPy.reshape(-1,1)

Test_Pred_COP = np.concatenate((Test_Pred_COPx, Test_Pred_COPy), axis=1)
Test_FC_COP = np.concatenate((Test_out_COPx, Test_out_COPy), axis=1)

Test_col_COP = 'COPx', 'COPy'

x=[]
colors=['red','green','brown','teal','gray','black','maroon','orange','purple']
colors2=['green','red','orange','black','maroon','teal','blue','gray','brown']
x = np.arange(0,2000)*40/2000
for i in range(0,2):
    plt.figure(figsize=(15,6))
    plt.plot(x,Test_FC_COP[0:2000,i], color='red')
    plt.plot(x,Test_Pred_COP[0:2000,i],markerfacecolor='none',color='green')
    plt.title('COP Calculation (Testing Data)')
    plt.ylabel(col_COP[i])
    plt.xlabel('Time(s)')
    plt.legend(['Groundtruth', 'Predicted Value'], loc='best')
    plt.savefig('Regression Result.png'[i])
    plt.show()

# Trajectory
from matplotlib import pyplot

x = range(50)
Test_y1 = Test_FC_COP[50:100,0]
Test_y2 = Test_FC_COP[50:100,1]
Test_y3 = Test_Pred_COP[50:100,0]
Test_y4 = Test_Pred_COP[50:100,1]

# pyplot.figure(figsize=(15,6))
# pyplot.plot(FC_COP[:,0],FC_COP[:,1])
# pyplot.show()

# Test_data_filter = abs(y1) > 0
# Test_data_filter2 = abs(y3) > 0
pyplot.figure(figsize=(15,6))
# pyplot.plot(Test_y1[Test_data_filter], Test_y2[Test_data_filter ], color='red', alpha=0.3)
# pyplot.plot(Test_y3[Test_data_filter2], Test_y4[Test_data_filter2 ], color='green')
pyplot.plot(Test_y1, Test_y2, color='red')
pyplot.plot(Test_y3, Test_y4, color='green')
plt.title('COP Trajectory (Testing Data)')
pyplot.ylabel('COPy (mm)')
pyplot.xlabel('COPx (mm)')
pyplot.legend(['Groundtruth', 'Predicted'], loc='best')
pyplot.show()

In [ ]:
x = range(100)
Test_y1 = Test_FC_COP[100:200,0]
Test_y2 = Test_FC_COP[100:200,1]
Test_y3 = Test_Pred_COP[100:200,0]
Test_y4 = Test_Pred_COP[100:200,1]

pyplot.figure(figsize=(15,6))
pyplot.plot(Test_y1, Test_y2, color='red')
pyplot.plot(Test_y3, Test_y4, color='green')
plt.title('COP Trajectory (Training Data)')
pyplot.ylabel('COPy (mm)')
pyplot.xlabel('COPx (mm)')
pyplot.legend(['Groundtruth', 'Predicted'], loc='best')
pyplot.show()

In [ ]:
x = range(100)
Test_y1 = Test_FC_COP[0:100,0]
Test_y2 = Test_FC_COP[0:100,1]
Test_y3 = Test_Pred_COP[0:100,0]
Test_y4 = Test_Pred_COP[0:100,1]

pyplot.figure(figsize=(15,6))
pyplot.plot(Test_y1, Test_y2, color='red')
pyplot.plot(Test_y3, Test_y4, color='green')
plt.title('COP Trajectory (Training Data)')
pyplot.ylabel('COPy (mm)')
pyplot.xlabel('COPx (mm)')
pyplot.legend(['Groundtruth', 'Predicted'], loc='best')
pyplot.show()